In [ ]:
# This script is to analyze genome coverage of mapping Illumina reads (Ug99, 210) against genome reference (Ug99 and 210) with high stringency parameters 
# Analyzing the coverage in sliding windows (i.e. 1000 bp) of the references using samtools bedcov.
# Authors: Benjamin Schwessinger and Feng Li

In [7]:
# set the PATH to the bioconda environment loaded

In [8]:
set_env PATH=/home/figueroa/lixx3627/.conda/envs/ben_anu/bin:/panfs/roc/msisoft/python3/3.4-conda/bin:/usr/local/bin:/usr/local/sbin:/usr/share/Modules/bin:/opt/moab/bin:/opt/rh/python33/root/usr/bin:/sbin:/usr/sbin:/bin:/usr/bin:/usr/local/bin:/usr/local/sbin:/opt/msi/bin:/opt/puppetlabs/bin:/home/figueroa/lixx3627/src/bioawk:/home/figueroa/lixx3627/src/seqtk/seqtk/

env: PATH=/home/figueroa/lixx3627/.conda/envs/ben_anu/bin:/panfs/roc/msisoft/python3/3.4-conda/bin:/usr/local/bin:/usr/local/sbin:/usr/share/Modules/bin:/opt/moab/bin:/opt/rh/python33/root/usr/bin:/sbin:/usr/sbin:/bin:/usr/bin:/usr/local/bin:/usr/local/sbin:/opt/msi/bin:/opt/puppetlabs/bin:/home/figueroa/lixx3627/src/bioawk:/home/figueroa/lixx3627/src/seqtk/seqtk/


In [9]:
%matplotlib inline

In [11]:
import pandas as pd
import os
import re
from Bio.SeqRecord import SeqRecord
import numpy as np
import pybedtools
import time
import matplotlib.pyplot as plt
import sys
import subprocess
import shutil
import glob
import scipy.stats as stats
import statsmodels as sms
import statsmodels.sandbox.stats.multicomp
import matplotlib
from sklearn.externals.joblib import Parallel, delayed
import itertools as it
from scipy.signal import argrelextrema
import scipy
from pybedtools import BedTool
import seaborn as sns
import altair as alt

In [82]:
def run_command(command):
    print('\nRunnning now!\n')
    print(command)
    output = subprocess.check_output(command, shell=True, stderr=subprocess.STDOUT)
    print('\nDone\nWith ouput:\n%s' % output)
    print(output)
    
def hello_print(command_list):
    print(command_list)

def df_to_tab(df, fn):
    """Takes a df and a filename to save it out as tab seperated file."""
    df.to_csv(fn, header=None, index=None, sep='\t')

def faidx_genome(genome_fn, contig_fn):
    '''Generates a genome file used for samtools and such if not already present.
    Input: 
    Abspath filename of output genome_file.
    Abspath filename for genome contig file.'''
    if not os.path.exists(genome_fn):
        samtools_command = 'samtools faidx %s' % contig_fn
        run_command(samtools_command)
        genome_file_command = 'cat %s.fai | sort -k1,1n | cut -f 1,2 > %s' %(contig_fn, genome_fn)
        run_command(genome_file_command)
    else:
        print('%s already exists!' % genome_fn)
    
def remove_files(file_list):
    for file in file_list:
        os.remove(file)

def run_sam_bedcov(bed_fn, bam_fn, samcov_fn, tmp_path, parallel = False):
    """This function runst samtools bedcov. It also has the option to do this in parallel spliting up the bed_cov file in equal junks plus one.
    Input:
        bed file name as Abspath.
        bam file name as Abspath.
        Samcov file as Abspath.
        Option to run thinks in parllel. Set to False. Provide the number of threads you want to run -1.
    Output:
        It generate the samcov files and saves them to file."""
    command = 'samtools bedcov %s %s > %s' % (bed_fn, bam_fn, samcov_fn)
    if parallel != False and type(parallel) == int:
        split_bed_files = bed_file_split(bed_fn, parallel, tmp_path)
        split_samcov_files = samcov_file_split(samcov_fn, len(split_bed_files), tmp_path=tmp_path)
        bam_files = [bam_fn]*len(split_bed_files)
        tmp_path_list = [tmp_path] * len(split_bed_files)
        n_threads = len(split_bed_files)
        Parallel(n_jobs=n_threads)(delayed(run_sam_bedcov)(bed, bam, sam, tmp_path) for bed, bam, sam in zip(split_bed_files, bam_files, split_samcov_files))
        combine_samcov_files(split_samcov_files)
        remove_files(split_samcov_files)
        remove_files(split_bed_files)
    else:
        run_command(command)

def bed_file_split(bed_fn, num_chunks, tmp_path):
    '''Splits a bedfile into even chunks of files plus the reads. Each file having the same number of lines.
    Input:
    bed file name abspath
    chunks to split into
    tmp_path where the split files get saved into.
    Returns:
    List of abspath filenames for split bed files.'''
    infilename = bed_fn
    number_of_lines = 0
    with open(infilename, 'rb') as infile:
        for x in infile:
            number_of_lines = number_of_lines + 1
    print('in_file_line_number:', number_of_lines)
    chunk_size = number_of_lines // num_chunks
    print('target chunk_size:', chunk_size)
    files = []
    
    previous = 0
    for i,y in enumerate([x for x in range(0, number_of_lines, chunk_size)]):
        if y == 0:
            continue
                    
        else:
            temp_fn = os.path.join(TMP_PATH, '%s_%s' % (os.path.basename(bed_fn), i-1))
            with open(temp_fn, 'w') as temp_file:
                with open(infilename, 'r') as infile:
                    for line_number, line in enumerate(infile):
                        if line_number < y and line_number >= previous:
                            print(line.rstrip(), file=temp_file)
                        else:
                            continue
            previous = y
            files.append(temp_fn)
    if y < number_of_lines:
        temp_fn = os.path.join(TMP_PATH, '%s_%s' % (os.path.basename(bed_fn), i))
        with open(temp_fn, 'w') as temp_file:
                with open(infilename, 'r') as infile:
                    for line_number, line in enumerate(infile):
                        if line_number >= y:
                            print(line.rstrip(), file=temp_file)
                        else:
                            continue
    files.append(temp_fn)
    return files

def samcov_file_split(basefile_name, chunks, tmp_path):
    """This function takes the samcov file and generates the split filenames necessary to run the 
    parallelized run_sam_bedcov."""
    chunk_base_name = os.path.join(tmp_path, os.path.basename(basefile_name))
    return_list = []
    for x in range(0, chunks):
        return_list.append('%s_%s' % (chunk_base_name, x))
    return return_list

def combine_samcov_files(file_list):
    """This function concatenates samcov files from a tmp folder into the folder below. And removes all tmp files."""
    command = 'cat'
    combined_file = os.path.basename(file_list[0])[:-2]
    combined_file_fn = os.path.join(os.path.dirname(file_list[0]),'..', combined_file)
    for file in file_list:
        command = command + ' %s' % file
    command = command + ' > %s'  % combined_file_fn
    run_command(command)
    

In [67]:
# #Define some PATH variables
# BASE_AA_PATH: reference genome fasta file; SRM_PATH: short-read mapping files
BASE_AA_PATH = '../../shared/Analysis/ug99_Jan/karyon_study/data/reference_genome/after_BP'
SRM_PATH = '../../shared/Analysis/ug99_Jan/karyon_study/analysis/mapped_reads/after_BP'
# COV_PATH: provide a PATH to write down covearge calculation (samcov) results
COV_PATH = '../../shared/Analysis/ug99_Jan/karyon_study/analysis/COV/'

# karyon assignment file for 210, Ug99, and 98 contain contig column and karyon assign column
KAYRON_ASSIGN_210 = '/panfs/roc/groups/7/figueroa/shared/Analysis/ug99_Jan/karyon_study/analysis/COV/karyon_assign_afterBP/210_karyon_assign.csv'
KAYRON_ASSIGN_Ug99 = '/panfs/roc/groups/7/figueroa/shared/Analysis/ug99_Jan/karyon_study/analysis/COV/karyon_assign_afterBP/Ug99_karyon_assign.csv'

# copy the samcov result files in another folder and gave it PATH as SAMCOV, when reusing samcov files for downstream analysis while not overwriting them
#SAMCOV = '/panfs/roc/groups/7/figueroa/shared/Analysis/ug99_Jan/karyon_study/analysis/COV/samcov/after_BP'

threads = 8

In [77]:
#OUT_PATH = os.path.join(os.path.abspath(COV_PATH), 'analysis', mapping_project)
OUT_PATH = os.path.join(os.path.abspath(COV_PATH))

TMP_PATH = os.path.join(OUT_PATH, 'tmp')
if not os.path.exists(TMP_PATH):
    os.makedirs(TMP_PATH)

In [69]:
# this block was used to define path names if I need to run samcov here

#define some filenames for assembly files
genome_list = [os.path.join(os.path.abspath(BASE_AA_PATH), x) for x in os.listdir(BASE_AA_PATH) if x.endswith('.fa') ]
genome_list.sort()
genome_file_fn_list = [x.replace('.fa', '.genome_file') for x in genome_list]
#and mapping bam files
bam_file_list = [os.path.join(os.path.abspath(SRM_PATH), x) for x in os.listdir(SRM_PATH) if x.endswith('.bam') ]

#generate the file names of the samecov files, when calculating samcov files
samcov_1k_file_list = [os.path.join(OUT_PATH, os.path.basename(x).replace('.bam', '.window_1k.samcov')) for x in bam_file_list]
# for SAMCOV PATH, when reusing samcov files
# samcov_1k_file_list = [os.path.join(os.path.abspath(SAMCOV), x) for x in os.listdir(SAMCOV) if x.endswith('1k.samcov') ]


In [70]:
genome_file_fn_list

['/panfs/roc/groups/7/figueroa/shared/Analysis/ug99_Jan/karyon_study/data/reference_genome/after_BP/210V2AP2_446ctg.genome_file',
 '/panfs/roc/groups/7/figueroa/shared/Analysis/ug99_Jan/karyon_study/data/reference_genome/after_BP/98V2AP2_333ctg.genome_file',
 '/panfs/roc/groups/7/figueroa/shared/Analysis/ug99_Jan/karyon_study/data/reference_genome/after_BP/Ug99V2AP2_537ctg.genome_file']

In [71]:
#now generate some genome files and some window files
for genome, genome_file in zip(genome_list, genome_file_fn_list):
    faidx_genome(genome_file, genome)



Runnning now!

samtools faidx /panfs/roc/groups/7/figueroa/shared/Analysis/ug99_Jan/karyon_study/data/reference_genome/after_BP/210V2AP2_446ctg.fa

Done
With ouput:
b''
b''

Runnning now!

cat /panfs/roc/groups/7/figueroa/shared/Analysis/ug99_Jan/karyon_study/data/reference_genome/after_BP/210V2AP2_446ctg.fa.fai | sort -k1,1n | cut -f 1,2 > /panfs/roc/groups/7/figueroa/shared/Analysis/ug99_Jan/karyon_study/data/reference_genome/after_BP/210V2AP2_446ctg.genome_file

Done
With ouput:
b''
b''

Runnning now!

samtools faidx /panfs/roc/groups/7/figueroa/shared/Analysis/ug99_Jan/karyon_study/data/reference_genome/after_BP/98V2AP2_333ctg.fa

Done
With ouput:
b''
b''

Runnning now!

cat /panfs/roc/groups/7/figueroa/shared/Analysis/ug99_Jan/karyon_study/data/reference_genome/after_BP/98V2AP2_333ctg.fa.fai | sort -k1,1n | cut -f 1,2 > /panfs/roc/groups/7/figueroa/shared/Analysis/ug99_Jan/karyon_study/data/reference_genome/after_BP/98V2AP2_333ctg.genome_file

Done
With ouput:
b''
b''
/panfs/roc/

In [72]:
#generate some window_files
#window_dict_1k2s = {} #the key is the genome name
window_dict_1k = {} #the key is the genome name
for genome_file in genome_file_fn_list:
    #generate the name for the window outfile
    key = os.path.basename(genome_file).split('.')[0]
    #out_name = os.path.join(OUT_PATH, os.path.basename(genome_file.replace('.genome_file', '.window_1k2s.bed')))
    #window_dict_1k2s[key] = out_name
    #now generate the window file and save out
    #BedTool().window_maker(g=genome_file, w=1000, s=200).saveas(out_name)
    #same thing for different window 1k without slide
    out_name = os.path.join(OUT_PATH, os.path.basename(genome_file.replace('.genome_file', '.window_1k.bed')))
    window_dict_1k[key] = out_name
    BedTool().window_maker(g=genome_file, w=1000).saveas(out_name)

In [73]:
window_dict_1k

{'210V2AP2_446ctg': '/panfs/roc/groups/7/figueroa/shared/Analysis/ug99_Jan/karyon_study/analysis/COV/210V2AP2_446ctg.window_1k.bed',
 '98V2AP2_333ctg': '/panfs/roc/groups/7/figueroa/shared/Analysis/ug99_Jan/karyon_study/analysis/COV/98V2AP2_333ctg.window_1k.bed',
 'Ug99V2AP2_537ctg': '/panfs/roc/groups/7/figueroa/shared/Analysis/ug99_Jan/karyon_study/analysis/COV/Ug99V2AP2_537ctg.window_1k.bed'}

In [74]:
samcov_1k_file_list

['/panfs/roc/groups/7/figueroa/shared/Analysis/ug99_Jan/karyon_study/analysis/COV/210V2AP2_446ctg.bwamem.210_WTN_clean_250PE_36M.window_1k.samcov',
 '/panfs/roc/groups/7/figueroa/shared/Analysis/ug99_Jan/karyon_study/analysis/COV/210V2AP2_446ctg.bwamem.98_WT_Clean_250PE_52M.window_1k.samcov',
 '/panfs/roc/groups/7/figueroa/shared/Analysis/ug99_Jan/karyon_study/analysis/COV/210V2AP2_446ctg.bwamem.Pgt_99_TTKSK_clean_150PE_43M.window_1k.samcov',
 '/panfs/roc/groups/7/figueroa/shared/Analysis/ug99_Jan/karyon_study/analysis/COV/98V2AP2_333ctg.bwamem.210_WTN_clean_250PE_36M.window_1k.samcov',
 '/panfs/roc/groups/7/figueroa/shared/Analysis/ug99_Jan/karyon_study/analysis/COV/98V2AP2_333ctg.bwamem.98_WT_Clean_250PE_52M.window_1k.samcov',
 '/panfs/roc/groups/7/figueroa/shared/Analysis/ug99_Jan/karyon_study/analysis/COV/98V2AP2_333ctg.bwamem.Pgt_99_TTKSK_clean_150PE_43M.window_1k.samcov',
 '/panfs/roc/groups/7/figueroa/shared/Analysis/ug99_Jan/karyon_study/analysis/COV/Ug99V2AP2_537ctg.bwamem.210

In [75]:
#generate the file names of the bedfiles
#bed_1k2s_file_list = [window_dict_1k2s[os.path.basename(x).split('.')[0]] for x in samcov_1k2s_file_list ]
bed_1k_file_list = [window_dict_1k[os.path.basename(x).split('.')[0]] for x in samcov_1k_file_list ]

In [83]:
#now run the window coverage analysis
for bed, bam, samcov in zip(bed_1k_file_list, bam_file_list, samcov_1k_file_list):
    run_sam_bedcov(bed, bam, samcov, tmp_path=TMP_PATH, parallel=threads)

in_file_line_number: 177238
target chunk_size: 22154

Runnning now!

Runnning now!

Runnning now!

Runnning now!

Runnning now!


samtools bedcov /panfs/roc/groups/7/figueroa/shared/Analysis/ug99_Jan/karyon_study/analysis/COV/tmp/210V2AP2_446ctg.window_1k.bed_0 /panfs/roc/groups/7/figueroa/shared/Analysis/ug99_Jan/karyon_study/analysis/mapped_reads/after_BP/210V2AP2_446ctg.bwamem.210_WTN_clean_250PE_36M.bam > /panfs/roc/groups/7/figueroa/shared/Analysis/ug99_Jan/karyon_study/analysis/COV/tmp/210V2AP2_446ctg.bwamem.210_WTN_clean_250PE_36M.window_1k.samcov_0

Runnning now!

samtools bedcov /panfs/roc/groups/7/figueroa/shared/Analysis/ug99_Jan/karyon_study/analysis/COV/tmp/210V2AP2_446ctg.window_1k.bed_1 /panfs/roc/groups/7/figueroa/shared/Analysis/ug99_Jan/karyon_study/analysis/mapped_reads/after_BP/210V2AP2_446ctg.bwamem.210_WTN_clean_250PE_36M.bam > /panfs/roc/groups/7/figueroa/shared/Analysis/ug99_Jan/karyon_study/analysis/COV/tmp/210V2AP2_446ctg.bwamem.210_WTN_clean_250PE_36M.window_

NameError: name 'combine_samcov_files' is not defined

In [10]:
columns_names = ['contig', 'start', 'stop' , 'cov']

In [11]:
# sort the samcov file list for easier query of files
samcov_1k_file_list.sort()
samcov_1k_file_list

['/panfs/roc/groups/7/figueroa/shared/Analysis/ug99_Jan/karyon_study/analysis/COV/samcov/after_BP/210V2AP2_446ctg.bwamem.210_WTN_clean_250PE_36M.window_1k.samcov',
 '/panfs/roc/groups/7/figueroa/shared/Analysis/ug99_Jan/karyon_study/analysis/COV/samcov/after_BP/210V2AP2_446ctg.bwamem.98_WT_Clean_250PE_52M.window_1k.samcov',
 '/panfs/roc/groups/7/figueroa/shared/Analysis/ug99_Jan/karyon_study/analysis/COV/samcov/after_BP/210V2AP2_446ctg.bwamem.Pgt_99_TTKSK_clean_150PE_43M.window_1k.samcov',
 '/panfs/roc/groups/7/figueroa/shared/Analysis/ug99_Jan/karyon_study/analysis/COV/samcov/after_BP/98V2AP2_333ctg.bwamem.210_WTN_clean_250PE_36M.window_1k.samcov',
 '/panfs/roc/groups/7/figueroa/shared/Analysis/ug99_Jan/karyon_study/analysis/COV/samcov/after_BP/98V2AP2_333ctg.bwamem.98_WT_Clean_250PE_52M.window_1k.samcov',
 '/panfs/roc/groups/7/figueroa/shared/Analysis/ug99_Jan/karyon_study/analysis/COV/samcov/after_BP/98V2AP2_333ctg.bwamem.Pgt_99_TTKSK_clean_150PE_43M.window_1k.samcov',
 '/panfs/roc/

In [12]:
samcov_1k_file_list[-1]

'/panfs/roc/groups/7/figueroa/shared/Analysis/ug99_Jan/karyon_study/analysis/COV/samcov/after_BP/Ug99V2AP2_537ctg.bwamem.Pgt_99_TTKSK_clean_150PE_43M.window_1k.samcov'

In [13]:
PgtUg99ref_rn_with_Ug99reads_df = pd.read_csv(samcov_1k_file_list[-1],sep='\t', header =None, names=columns_names)

In [14]:
# ug99 reads to ug99 assembly: Ug99V2AP2_renamed.bwamem.Pgt_99_TTKSK_clean_150PE_43M.window_1k.samcov- 1k
PgtUg99ref_rn_with_Ug99reads_df['cov_per_base'] = \
PgtUg99ref_rn_with_Ug99reads_df['cov']/(PgtUg99ref_rn_with_Ug99reads_df['stop']- PgtUg99ref_rn_with_Ug99reads_df['start'])
print(samcov_1k_file_list[-1])

/panfs/roc/groups/7/figueroa/shared/Analysis/ug99_Jan/karyon_study/analysis/COV/samcov/after_BP/Ug99V2AP2_537ctg.bwamem.Pgt_99_TTKSK_clean_150PE_43M.window_1k.samcov


In [15]:
# mean coverage
PgtUg99ref_rn_with_Ug99reads_df['cov_per_base'].mean()

31.9620062435209

In [16]:
#naming is similar to my mapping naming where reference is first and reads come second
# map 210 reads to ug99
PgtUg99ref_rn_with_210reads_df = pd.read_csv(samcov_1k_file_list[-3],sep='\t', header =None, names=columns_names)
PgtUg99ref_rn_with_210reads_df['cov_per_base'] = \
PgtUg99ref_rn_with_210reads_df['cov']/(PgtUg99ref_rn_with_210reads_df['stop']- PgtUg99ref_rn_with_210reads_df['start'])


In [17]:
##Now read in the karyon assignment
#read ug99 karyon assignment data - the master excel worksheet
df_karyon_assigment = pd.read_csv(KAYRON_ASSIGN_Ug99)

In [18]:
df_karyon_assigment.tail()

,contig_id,chr,Name2,chrlen.x,BinAsign,telomere_pos,cov_ug99v2,cov_Leftover_ug99v2,karyon_assign
532,Ug99V2_tig00016875_02,tig00016875_pilon_pilon,tig00016875_02,197440,Bin46,NaN,NaN,NaN,A
533,Ug99V2_tig00000037_01,tig00000037_pilon_pilon,tig00000037_01,244068,Bin49,NaN,NaN,NaN,A
534,Ug99V2_tig00000037_02,tig00000037_pilon_pilon,tig00000037_02,425610,Bin49,NaN,NaN,NaN,C
535,Ug99V2_tig00000316_01,tig00000316_pilon_pilon,tig00000316_01,44000,NaN,NaN,NaN,NaN,C
536,Ug99V2_tig00000316_02,tig00000316_pilon_pilon,tig00000316_02,86949,NaN,NaN,NaN,NaN,A


In [19]:
Pgt_ug99_karyon_dict = dict(zip(df_karyon_assigment['contig_id'], df_karyon_assigment['karyon_assign']))

In [20]:
Pgt_ug99_karyon_dict["Ug99V2_tig00000316_02"]

'A'

In [21]:
PgtUg99ref_rn_with_Ug99reads_df.tail()

,contig,start,stop,cov,cov_per_base
176501,Ug99V2_tig00016883,284000,285000,10014,10.014
176502,Ug99V2_tig00016883,285000,286000,12072,12.072
176503,Ug99V2_tig00016883,286000,287000,5928,5.928
176504,Ug99V2_tig00016883,287000,288000,0,0.000
176505,Ug99V2_tig00016883,288000,288056,0,0.000


In [22]:
# append another column containing the karyon assignment info into the samcov.df
PgtUg99ref_rn_with_Ug99reads_df['karyon_assign'] = \
PgtUg99ref_rn_with_Ug99reads_df['contig'].apply(lambda x: Pgt_ug99_karyon_dict[x])

In [23]:
PgtUg99ref_rn_with_Ug99reads_df['contig'][0]

'Ug99V2_tig00000005'

In [24]:
Pgt_ug99_karyon_dict[PgtUg99ref_rn_with_Ug99reads_df['contig'][0]]

'C'

In [25]:
PgtUg99ref_rn_with_Ug99reads_df.tail()

,contig,start,stop,cov,cov_per_base,karyon_assign
176501,Ug99V2_tig00016883,284000,285000,10014,10.014,C
176502,Ug99V2_tig00016883,285000,286000,12072,12.072,C
176503,Ug99V2_tig00016883,286000,287000,5928,5.928,C
176504,Ug99V2_tig00016883,287000,288000,0,0.000,C
176505,Ug99V2_tig00016883,288000,288056,0,0.000,C


In [26]:
#get the mt contig to remove 
PgtUg99ref_rn_with_Ug99reads_df[PgtUg99ref_rn_with_Ug99reads_df['karyon_assign'] == 'mt'].index

Int64Index([55818, 55819, 55820, 55821, 55822, 55823, 55824, 55825, 55826,
            55827,
            ...
            55915, 55916, 55917, 55918, 55919, 55920, 55921, 55922, 55923,
            55924],
           dtype='int64', length=107)

In [27]:
# remove mt
PgtUg99ref_rn_with_Ug99reads_df.drop\
(PgtUg99ref_rn_with_Ug99reads_df[PgtUg99ref_rn_with_Ug99reads_df['karyon_assign'] == 'mt'].index, inplace = True)

# remove ?
PgtUg99ref_rn_with_Ug99reads_df.drop\
(PgtUg99ref_rn_with_Ug99reads_df[PgtUg99ref_rn_with_Ug99reads_df['karyon_assign'] == '?'].index, inplace = True)

In [28]:
colors = {'A': '#95d0fc', 'C': '#15b01a'}


In [29]:
# 210 reads to Ug99

In [30]:
# append another column containing the karyon assignment info into the samcov.df
PgtUg99ref_rn_with_210reads_df['karyon_assign'] = \
PgtUg99ref_rn_with_210reads_df['contig'].apply(lambda x: Pgt_ug99_karyon_dict[x])

# remove mt
PgtUg99ref_rn_with_210reads_df.drop\
(PgtUg99ref_rn_with_210reads_df[PgtUg99ref_rn_with_210reads_df['karyon_assign'] == 'mt'].index, inplace = True)

# remove ?
PgtUg99ref_rn_with_210reads_df.drop\
(PgtUg99ref_rn_with_210reads_df[PgtUg99ref_rn_with_210reads_df['karyon_assign'] == '?'].index, inplace = True)

In [31]:
# normalization

In [32]:
# normalize by the mean of karyon each karyon
PgtUg99ref_rn_with_Ug99reads_df['norm_cov_by_mean_of_karyon'] = 0
for kayron in PgtUg99ref_rn_with_Ug99reads_df['karyon_assign'].unique():
    _index = PgtUg99ref_rn_with_Ug99reads_df[PgtUg99ref_rn_with_Ug99reads_df['karyon_assign'] == kayron].index
    PgtUg99ref_rn_with_Ug99reads_df.loc[_index,'norm_cov_by_mean_of_karyon'] = \
    PgtUg99ref_rn_with_Ug99reads_df.loc[_index,'cov_per_base']/PgtUg99ref_rn_with_Ug99reads_df.loc[_index, 'cov_per_base'].mean()
    

In [33]:
PgtUg99ref_rn_with_Ug99reads_df.to_csv("PgtUg99ref_rn_with_Ug99reads_df.csv")

In [35]:
# normalize by the mean of karyon 
PgtUg99ref_rn_with_210reads_df['norm_cov_by_mean_of_karyon'] = 0
for kayron in PgtUg99ref_rn_with_210reads_df['karyon_assign'].unique():
    _index = PgtUg99ref_rn_with_210reads_df[PgtUg99ref_rn_with_210reads_df['karyon_assign'] == kayron].index
    PgtUg99ref_rn_with_210reads_df.loc[_index,'norm_cov_by_mean_of_karyon'] = \
    PgtUg99ref_rn_with_210reads_df.loc[_index,'cov_per_base']/PgtUg99ref_rn_with_210reads_df.loc[_index, 'cov_per_base'].mean()
    

In [36]:
PgtUg99ref_rn_with_210reads_df.to_csv("PgtUg99ref_rn_with_210reads_df.csv")

In [38]:
#210 reads to 210

In [39]:
samcov_1k_file_list[0]

'/panfs/roc/groups/7/figueroa/shared/Analysis/ug99_Jan/karyon_study/analysis/COV/samcov/after_BP/210V2AP2_446ctg.bwamem.210_WTN_clean_250PE_36M.window_1k.samcov'

In [40]:
print(samcov_1k_file_list[0])
Pgt210ref_rn_with_210reads_df = pd.read_csv(samcov_1k_file_list[0],sep='\t', header =None, names=columns_names)
# 210 reads to 210 assembly: 
Pgt210ref_rn_with_210reads_df['cov_per_base'] = \
Pgt210ref_rn_with_210reads_df['cov']/(Pgt210ref_rn_with_210reads_df['stop']- Pgt210ref_rn_with_210reads_df['start'])


/panfs/roc/groups/7/figueroa/shared/Analysis/ug99_Jan/karyon_study/analysis/COV/samcov/after_BP/210V2AP2_446ctg.bwamem.210_WTN_clean_250PE_36M.window_1k.samcov


In [41]:
# map ug99 reads to 210

In [42]:
#naming is similar to my mapping naming where reference is first and reads come second
print(samcov_1k_file_list[2])
Pgt210ref_rn_with_ug99reads_df = pd.read_csv(samcov_1k_file_list[2],sep='\t', header =None, names=columns_names)
Pgt210ref_rn_with_ug99reads_df['cov_per_base'] = \
Pgt210ref_rn_with_ug99reads_df['cov']/(Pgt210ref_rn_with_ug99reads_df['stop']- Pgt210ref_rn_with_ug99reads_df['start'])

/panfs/roc/groups/7/figueroa/shared/Analysis/ug99_Jan/karyon_study/analysis/COV/samcov/after_BP/210V2AP2_446ctg.bwamem.Pgt_99_TTKSK_clean_150PE_43M.window_1k.samcov


In [43]:
##Now read in the karyon assignment
#read ug99 karyon assignment data - the master excel worksheet
df_karyon_assigment_210 = pd.read_csv(KAYRON_ASSIGN_210)
df_karyon_assigment_210.head()

,contig_id,Sample Name_afterABbreak,Sample Name_ori,Bin_assignment,Length,LengthN,Sample Name short,210WTCov_of_mappingTo210V2AP2,210V2mappingCov_of_unmappedReadsFromUg99V2mapping,karyonAssign,LC<1X,notes
0,210V2_tig00000081,tig00000081,tig00000081_arrow_pilon_pilon,210v2aP2_tigs_not binned,45646,45646,tig00000081,6,2,?,Y,repeat
1,210V2_tig00000154,tig00000154,tig00000154_arrow_pilon_pilon_3Tel_TTAGGG,210v2aP2_tigs_not binned,32135,32135,tig00000154,33,6,?,NaN,NaN
2,210V2_tig00000231,tig00000231,tig00000231_arrow_pilon_pilon,210v2aP2_tigs_not binned,60236,60236,tig00000231,343,6,?,NaN,repeat
3,210V2_tig00000235,tig00000235,tig00000235_arrow_pilon_pilon,210v2aP2_tigs_not binned,30751,30751,tig00000235,401,1,?,NaN,repeat
4,210V2_tig00000243,tig00000243,tig00000243_arrow_pilon_pilon,210v2aP2_tigs_not binned,44669,44669,tig00000243,372,1,?,NaN,repeat


In [44]:
Pgt_210_karyon_dict = dict(zip(df_karyon_assigment_210['contig_id'], df_karyon_assigment_210['karyonAssign']))

In [45]:
# append another column containing the karyon assignment info into the samcov.df
Pgt210ref_rn_with_210reads_df['karyon_assign'] = \
Pgt210ref_rn_with_210reads_df['contig'].apply(lambda x: Pgt_210_karyon_dict[x])

In [46]:
Pgt210ref_rn_with_210reads_df.head()

,contig,start,stop,cov,cov_per_base,karyon_assign
0,210V2_tig00000006,0,1000,16124,16.124,A
1,210V2_tig00000006,1000,2000,19194,19.194,A
2,210V2_tig00000006,2000,3000,61,0.061,A
3,210V2_tig00000006,3000,4000,1976,1.976,A
4,210V2_tig00000006,4000,5000,21,0.021,A


In [47]:
# 210 reads to 210 reads
# remove mt
Pgt210ref_rn_with_210reads_df.drop\
(Pgt210ref_rn_with_210reads_df[Pgt210ref_rn_with_210reads_df['karyon_assign'] == 'mt'].index, inplace = True)
# remove ? 
Pgt210ref_rn_with_210reads_df.drop\
(Pgt210ref_rn_with_210reads_df[Pgt210ref_rn_with_210reads_df['karyon_assign'] == '?'].index, inplace = True)

In [48]:
# ug99 reads to 210

In [49]:
# append another column containing the karyon assignment info into the samcov.df
Pgt210ref_rn_with_ug99reads_df['karyon_assign'] = \
Pgt210ref_rn_with_ug99reads_df['contig'].apply(lambda x: Pgt_210_karyon_dict[x])

# remove mt
Pgt210ref_rn_with_ug99reads_df.drop\
(Pgt210ref_rn_with_ug99reads_df[Pgt210ref_rn_with_ug99reads_df['karyon_assign'] == 'mt'].index, inplace = True)
# remove ? 
Pgt210ref_rn_with_ug99reads_df.drop\
(Pgt210ref_rn_with_ug99reads_df[Pgt210ref_rn_with_ug99reads_df['karyon_assign'] == '?'].index, inplace = True)


In [50]:
# normalize by the mean of karyon 
Pgt210ref_rn_with_210reads_df['norm_cov_by_mean_of_karyon'] = 0
for kayron in Pgt210ref_rn_with_210reads_df['karyon_assign'].unique():
    _index = Pgt210ref_rn_with_210reads_df[Pgt210ref_rn_with_210reads_df['karyon_assign'] == kayron].index
    Pgt210ref_rn_with_210reads_df.loc[_index,'norm_cov_by_mean_of_karyon'] = \
    Pgt210ref_rn_with_210reads_df.loc[_index,'cov_per_base']/Pgt210ref_rn_with_210reads_df.loc[_index, 'cov_per_base'].mean()
    

In [51]:
Pgt210ref_rn_with_210reads_df.to_csv("Pgt210ref_rn_with_210reads_df.csv")

In [52]:
#colors2 = {'A': '#95d0fc', 'B': '#ffb07c'}
colors2_210 = {'A': '#95d0fc', 'B': '#ffb07c'}

In [54]:
# normalize by the mean of karyon
Pgt210ref_rn_with_ug99reads_df['norm_cov_by_mean_of_karyon'] = 0
for kayron in Pgt210ref_rn_with_ug99reads_df['karyon_assign'].unique():
    _index = Pgt210ref_rn_with_ug99reads_df[Pgt210ref_rn_with_ug99reads_df['karyon_assign'] == kayron].index
    Pgt210ref_rn_with_ug99reads_df.loc[_index,'norm_cov_by_mean_of_karyon'] = \
    Pgt210ref_rn_with_ug99reads_df.loc[_index,'cov_per_base']/Pgt210ref_rn_with_ug99reads_df.loc[_index, 'cov_per_base'].mean()

In [55]:
Pgt210ref_rn_with_ug99reads_df.to_csv("Pgt210ref_rn_with_ug99reads_df.csv")